In [1]:
import pandas as pd
data = pd.read_csv('inverter Data Set.csv')


In [2]:
dfx1 = data.iloc[:,0:-3]
#dfy1 = data_raw[['u_a_k-1','u_b_k-1','u_c_k-1']]
dfy1 = data.iloc[:,-3:-2]


In [6]:
from sklearn.model_selection import train_test_split    
X_train, X_test1, y_train, y_test = train_test_split(dfx1, dfy1, test_size=0.30, shuffle=False)


In [4]:
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor

# PSO Parameters
class Particle:
    def __init__(self, dim, bounds):
        self.position = np.array([random.uniform(bounds[d][0], bounds[d][1]) for d in range(dim)])
        self.velocity = np.zeros(dim)
        self.best_position = np.copy(self.position)
        self.best_score = float('inf')

def pso_optimize(func, dim, bounds, n_particles=10, max_iter=30, w=0.5, c1=1.5, c2=1.5, early_stop=5):
    particles = [Particle(dim, bounds) for _ in range(n_particles)]
    global_best = min(particles, key=lambda p: func(p.position))
    
    best_loss = func(global_best.position)
    no_improve_count = 0
    
    for _ in range(max_iter):
        for particle in particles:
            score = func(particle.position)
            if score < particle.best_score:
                particle.best_score = score
                particle.best_position = np.copy(particle.position)
            
            if score < best_loss:
                global_best = particle
                best_loss = score
                no_improve_count = 0
            else:
                no_improve_count += 1
            
            # Early stopping
            if no_improve_count >= early_stop:
                return global_best.position, best_loss
            
            # Update velocity and position
            inertia = w * particle.velocity
            cognitive = c1 * random.random() * (particle.best_position - particle.position)
            social = c2 * random.random() * (global_best.position - particle.position)
            particle.velocity = inertia + cognitive + social
            particle.position += particle.velocity
            
    return global_best.position, best_loss

# PSO for Regression Models
def regression_pso(model_class, param_bounds, n_particles=5, max_iter=15):
    y =np.array(Y_test).ravel()
    X_train, X_test, y_train, y_test = train_test_split(X_test1, y, test_size=0.2, random_state=42)
    
    def model_fitness(params):
        params = {key: int(val) if isinstance(param_bounds[key][0], int) else val for key, val in zip(param_bounds.keys(), params)}
        model = model_class(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        return mean_squared_error(y_test, y_pred)
    
    bounds = list(param_bounds.values())
    best_params, best_loss = pso_optimize(model_fitness, len(bounds), bounds, n_particles, max_iter)
    
    best_params_dict = {key: int(val) if isinstance(param_bounds[key][0], int) else val for i, (key, val) in enumerate(zip(param_bounds.keys(), best_params))}
    return best_params_dict, best_loss

if __name__ == "__main__":
    models = {
        # "DecisionTree": (DecisionTreeRegressor, {"max_depth": (1, 10), "min_samples_split": (2, 10)}),
        "GradientBoosting": (GradientBoostingRegressor, {"n_estimators": (50, 100), "learning_rate": (0.05, 0.2)}),
        "XGBRegressor": (XGBRegressor, {"n_estimators": (50, 100), "learning_rate": (0.05, 0.2)}),
        "RandomForest": (RandomForestRegressor, {"n_estimators": (50, 100), "max_depth": (1, 10)})
    }
    
    for name, (model_class, param_bounds) in models.items():
        print(f"Optimizing {name} using PSO...")
        best_params, best_loss = regression_pso(model_class, param_bounds)
        print(f"Best Parameters: {best_params}, Best Loss: {best_loss}\n")

Optimizing GradientBoosting using PSO...


KeyboardInterrupt: 

In [ ]:
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from joblib import Parallel, delayed  # For parallel processing

# Particle class for PSO
class Particle:
    def __init__(self, dim, bounds):
        self.position = np.array([random.uniform(bounds[d][0], bounds[d][1]) for d in range(dim)])
        self.velocity = np.zeros(dim)
        self.best_position = np.copy(self.position)
        self.best_score = float('inf')

def pso_optimize(func, dim, bounds, n_particles=7, max_iter=20, w=0.5, c1=1.5, c2=1.5, early_stop=5):
    particles = [Particle(dim, bounds) for _ in range(n_particles)]
    global_best = min(particles, key=lambda p: func(p.position))

    best_loss = func(global_best.position)
    no_improve_count = 0

    for _ in range(max_iter):
        # Parallel evaluation of particle fitness
        scores = Parallel(n_jobs=-1)(delayed(func)(p.position) for p in particles)

        for i, particle in enumerate(particles):
            score = scores[i]

            if score < particle.best_score:
                particle.best_score = score
                particle.best_position = np.copy(particle.position)

            if score < best_loss:
                global_best = particle
                best_loss = score
                no_improve_count = 0
            else:
                no_improve_count += 1

            # Early stopping
            if no_improve_count >= early_stop:
                return global_best.position, best_loss

            # Update velocity and position
            inertia = w * particle.velocity
            cognitive = c1 * random.random() * (particle.best_position - particle.position)
            social = c2 * random.random() * (global_best.position - particle.position)
            particle.velocity = inertia + cognitive + social

            # Update position and ensure it stays within bounds
            particle.position += particle.velocity
            particle.position = np.clip(particle.position, [b[0] for b in bounds], [b[1] for b in bounds])

    return global_best.position, best_loss

# Function to optimize regression model using PSO
def regression_pso(model_class, param_bounds, X_train, X_test, y_train, y_test, n_particles=7, max_iter=12):
    def model_fitness(params):
        # Convert parameters to correct types (int for discrete values)
        params = {key: int(val) if isinstance(param_bounds[key][0], int) else val for key, val in zip(param_bounds.keys(), params)}
        model = model_class(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        return mean_squared_error(y_test, y_pred)

    bounds = list(param_bounds.values())
    best_params, best_loss = pso_optimize(model_fitness, len(bounds), bounds, n_particles, max_iter)

    # Convert best parameters to correct types
    best_params_dict = {key: int(val) if isinstance(param_bounds[key][0], int) else val for key, val in zip(param_bounds.keys(), best_params)}
    return best_params_dict, best_loss

# Generate sample regression data
X, Y = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=42)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

if __name__ == "__main__":
    models = {
        "GradientBoosting": (GradientBoostingRegressor, {"n_estimators": (50, 100), "learning_rate": (0.05, 0.2)}),
        "XGBRegressor": (XGBRegressor, {"n_estimators": (50, 100), "learning_rate": (0.05, 0.2)}),
        "RandomForest": (RandomForestRegressor, {"n_estimators": (50, 100), "max_depth": (1, 10)})
    }

    for name, (model_class, param_bounds) in models.items():
        print(f"Optimizing {name} using PSO...")
        best_params, best_loss = regression_pso(model_class, param_bounds, X_train, X_test, Y_train, Y_test)
        print(f"Best Parameters: {best_params}, Best Loss: {best_loss}\n")


Optimizing GradientBoosting using PSO...
Best Parameters: {'n_estimators': 95, 'learning_rate': 0.18068326107174232}, Best Loss: 966.7597823129357

Optimizing XGBRegressor using PSO...
Best Parameters: {'n_estimators': 67, 'learning_rate': 0.11974981294015839}, Best Loss: 1506.2004467484524

Optimizing RandomForest using PSO...
Best Parameters: {'n_estimators': 67, 'max_depth': 8}, Best Loss: 2754.9013856246956



In [8]:
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from joblib import Parallel, delayed  # For parallel processing

# Particle class for PSO
class Particle:
    def __init__(self, dim, bounds):
        self.position = np.array([random.uniform(bounds[d][0], bounds[d][1]) for d in range(dim)])
        self.velocity = np.zeros(dim)
        self.best_position = np.copy(self.position)
        self.best_score = float('inf')

def pso_optimize(func, dim, bounds, n_particles=7, max_iter=20, w=0.5, c1=1.5, c2=1.5, early_stop=5):
    particles = [Particle(dim, bounds) for _ in range(n_particles)]
    global_best = min(particles, key=lambda p: func(p.position))

    best_loss = func(global_best.position)
    no_improve_count = 0

    for _ in range(max_iter):
        # Parallel evaluation of particle fitness
        scores = Parallel(n_jobs=-1)(delayed(func)(p.position) for p in particles)

        for i, particle in enumerate(particles):
            score = scores[i]

            if score < particle.best_score:
                particle.best_score = score
                particle.best_position = np.copy(particle.position)

            if score < best_loss:
                global_best = particle
                best_loss = score
                no_improve_count = 0
            else:
                no_improve_count += 1

            # Early stopping
            if no_improve_count >= early_stop:
                return global_best.position, best_loss

            # Update velocity and position
            inertia = w * particle.velocity
            cognitive = c1 * random.random() * (particle.best_position - particle.position)
            social = c2 * random.random() * (global_best.position - particle.position)
            particle.velocity = inertia + cognitive + social

            # Update position and ensure it stays within bounds
            particle.position += particle.velocity
            particle.position = np.clip(particle.position, [b[0] for b in bounds], [b[1] for b in bounds])

    return global_best.position, best_loss

# Function to optimize regression model using PSO
def regression_pso(model_class, param_bounds, X_train, X_test, y_train, y_test, n_particles=7, max_iter=12):
    def model_fitness(params):
        # Convert parameters to correct types (int for discrete values)
        params = {key: int(val) if isinstance(param_bounds[key][0], int) else val for key, val in zip(param_bounds.keys(), params)}
        model = model_class(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        return mean_squared_error(y_test, y_pred)

    bounds = list(param_bounds.values())
    best_params, best_loss = pso_optimize(model_fitness, len(bounds), bounds, n_particles, max_iter)

    # Convert best parameters to correct types
    best_params_dict = {key: int(val) if isinstance(param_bounds[key][0], int) else val for key, val in zip(param_bounds.keys(), best_params)}
    return best_params_dict, best_loss

# Generate sample regression data
# X, Y = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=42)
y = np.array(y_test).ravel()
X_train, X_test, Y_train, Y_test = train_test_split(X_test1, y, test_size=0.2, random_state=42)

if __name__ == "__main__":
    models = {
        "GradientBoosting": (GradientBoostingRegressor, {"n_estimators": (50, 100), "learning_rate": (0.05, 0.2)}),
        "XGBRegressor": (XGBRegressor, {"n_estimators": (50, 100), "learning_rate": (0.05, 0.2)}),
        "RandomForest": (RandomForestRegressor, {"n_estimators": (50, 100), "max_depth": (1, 10)})
    }

    for name, (model_class, param_bounds) in models.items():
        print(f"Optimizing {name} using PSO...")
        best_params, best_loss = regression_pso(model_class, param_bounds, X_train, X_test, Y_train, Y_test)
        print(f"Best Parameters: {best_params}, Best Loss: {best_loss}\n")


Optimizing GradientBoosting using PSO...
Best Parameters: {'n_estimators': 96, 'learning_rate': 0.19104527704688606}, Best Loss: 3.744757125329138

Optimizing XGBRegressor using PSO...
Best Parameters: {'n_estimators': 98, 'learning_rate': 0.12702909825189335}, Best Loss: 0.7779491026847029

Optimizing RandomForest using PSO...
Best Parameters: {'n_estimators': 82, 'max_depth': 9}, Best Loss: 1.8720939753949268



In [11]:
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from joblib import Parallel, delayed  # For parallel processing

# Particle class for PSO
class Particle:
    def __init__(self, dim, bounds):
        self.position = np.array([random.uniform(bounds[d][0], bounds[d][1]) for d in range(dim)])
        self.velocity = np.zeros(dim)
        self.best_position = np.copy(self.position)
        self.best_score = float('inf')

def pso_optimize(func, dim, bounds, n_particles=7, max_iter=20, w=0.5, c1=1.5, c2=1.5, early_stop=5):
    particles = [Particle(dim, bounds) for _ in range(n_particles)]
    global_best = min(particles, key=lambda p: func(p.position))

    best_loss = func(global_best.position)
    no_improve_count = 0

    for _ in range(max_iter):
        # Parallel evaluation of particle fitness
        scores = Parallel(n_jobs=-1)(delayed(func)(p.position) for p in particles)

        for i, particle in enumerate(particles):
            score = scores[i]

            if score < particle.best_score:
                particle.best_score = score
                particle.best_position = np.copy(particle.position)

            if score < best_loss:
                global_best = particle
                best_loss = score
                no_improve_count = 0
            else:
                no_improve_count += 1

            # Early stopping
            if no_improve_count >= early_stop:
                return global_best.position, best_loss

            # Update velocity and position
            inertia = w * particle.velocity
            cognitive = c1 * random.random() * (particle.best_position - particle.position)
            social = c2 * random.random() * (global_best.position - particle.position)
            particle.velocity = inertia + cognitive + social

            # Update position and ensure it stays within bounds
            particle.position += particle.velocity
            particle.position = np.clip(particle.position, [b[0] for b in bounds], [b[1] for b in bounds])

    return global_best.position, best_loss

# Function to optimize regression model using PSO
def regression_pso(model_class, param_bounds, X_train, X_test, y_train, y_test, n_particles=7, max_iter=12):
    def model_fitness(params):
        # Convert parameters to correct types (int for discrete values)
        params = {key: int(val) if isinstance(param_bounds[key][0], int) else val for key, val in zip(param_bounds.keys(), params)}
        model = model_class(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        return mean_squared_error(y_test, y_pred)

    bounds = list(param_bounds.values())
    best_params, best_loss = pso_optimize(model_fitness, len(bounds), bounds, n_particles, max_iter)

    # Convert best parameters to correct types
    best_params_dict = {key: int(val) if isinstance(param_bounds[key][0], int) else val for key, val in zip(param_bounds.keys(), best_params)}
    return best_params_dict, best_loss

# Generate sample regression data
# X, Y = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=42)
y = np.array(y_test).ravel()
X_train, X_test, Y_train, Y_test = train_test_split(X_test1, y, test_size=0.2, random_state=42)

if __name__ == "__main__":
    models = {
        # "GradientBoosting": (GradientBoostingRegressor, {"n_estimators": (50, 100), "learning_rate": (0.05, 0.2)}),
        # "XGBRegressor": (XGBRegressor, {"n_estimators": (50, 100), "learning_rate": (0.05, 0.2)}),
        "RandomForest": (RandomForestRegressor, {"n_estimators": (50, 100), "max_depth": (1, 20)}),
        
        "DecisionTree": (DecisionTreeRegressor, {"max_depth": (1, 20), "min_samples_split": (2, 10)}),
    }

    for name, (model_class, param_bounds) in models.items():
        print(f"Optimizing {name} using PSO...")
        best_params, best_loss = regression_pso(model_class, param_bounds, X_train, X_test, Y_train, Y_test)
        print(f"Best Parameters: {best_params}, Best Loss: {best_loss}\n")


Optimizing RandomForest using PSO...
Best Parameters: {'n_estimators': 93, 'max_depth': 18}, Best Loss: 0.34348393013289963

Optimizing DecisionTree using PSO...
Best Parameters: {'max_depth': 16, 'min_samples_split': 2}, Best Loss: 0.9304527817225453

